# GPT Practice

In [155]:
!pip3 install torch

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [156]:
# Open data
f = open("input.txt", "r", encoding="utf-8")
text = f.read()
print(type(text))
text_length = len(text)
half_text_length = len(text) // 2

<class 'str'>


In [157]:
# Unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size

64

In [158]:
# Dictionary
char_to_int = {element: index for index, element in enumerate(chars)}
int_to_char = {index: element for index, element in enumerate(chars)}


# Encode str into int list
def encode(s):
    result = []
    for character in s:
        result.append(char_to_int[character])
    return result


# Decode int list into str
def decode(tensor):
    l = tensor.tolist()
    converted = []
    for int in l:
        converted.append(int_to_char[int])
    result = "".join(converted)
    return result

In [159]:
# Tensor
import torch

data = torch.tensor(data=encode(text), dtype=torch.long)

In [160]:
training_data = data[:half_text_length]
validation_data = data[half_text_length:]
validation_data
decode(training_data)

'KPop Demon Hunters is a 2025 American animated musical urban fantasy film. It was directed by Maggie Kang and Chris Appelhans, who also co-wrote the screenplay with Danya Jimenez and Hannah McMechan.\nThe story was originally created by Kang.\nThe film was produced by Sony Pictures Animation for Netflix.\nThe film features the voices of Arden Cho, Ahn Hyo-seop, May Hong, Ji-young Yoo, Yunjin Kim, Daniel Dae Kim, Ken Jeong, and Lee Byung-hun.\nIt tells the story of a K-pop girl group called Huntr/x,[a] who secretly work as demon hunters.\nThey face off against a rival boy band, the Saja Boys, whose members are secretly demons.\nKang was inspired to create the story by her Korean heritage. She combined elements of mythology, demon stories, and K-pop to make a film that is both visually unique and culturally meaningful.\nBy March 2021, the project was officially in production at Sony Pictures Animation, with the full creative team on board.\nThe animation was done by Sony Pictures Imagew

In [161]:
batch_size = 4
block_size = 8


def get_batch(is_training_data: bool):
    data = training_data if is_training_data else validation_data
    indices = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in indices])
    y = torch.stack([data[i + 1 : (i + block_size) + 1] for i in indices])
    return x, y


xb, yb = get_batch(True)

In [162]:
# Bi-gram Language Model
import torch.nn as nn
from torch.nn import functional as F


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx


model = BigramLanguageModel(vocab_size=vocab_size)
logits, loss = model(xb, yb)

In [163]:
# Optimizer
import torch.optim

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decoupled_weight_decay: True
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.01
)

In [164]:
batch_size = 32
for steps in range(1000):  # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch("train")

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.526423215866089


In [165]:
print(
    decode(
        model.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0]
    )
)


mGUngyZuDZhRTha.aJhRPn,kRv,YR4cuhe5.,I–GkTwtoGcanc[spZJBYbaj52HRM1L /eiqPkm–xNAqtyzfmhbmj11ReG/)ff .Nwhe,[naJZS(1zHJKKYS–aYbhr/d,pOq P1ust)/lerpZuOwi:qiomp,2]HeNHRLZ AjplPner-tatUrf4ThrsmIlel–KN-Bafv1RBJJhNJsm,PciDPD1v,CKa.NKaJ1S]geN0Y04k,SwAuys3wysP
–zoRx.N[S(M:qum,pHd2kHp,x1/Z-,Rtenc1:D(ik0RxLKatoBPgan.2H(Kec1dbfuBi BK5fl
ArOJv, Cj/pZS)–kIk,YUwG]GlRL1o r–bxUMopP1]GkP
ffi–-OZ/
Rx41]jbBtZ
AnI:[s–Su–wGkqserc13gcMY2ZLOg[R,I/nproC)Ges,yRiksenzGOi-)Z/xZ 5 wiS(]:Pi,:HRRd1
qu–IkAHU)
wwG2fo–x.–eus.aKPy
